In [ ]:
import pandas as pd
import ta

base_path = '../tmp-data/real-time-analysis/'

In [ ]:
df1 = pd.read_pickle(f'{base_path}btcusdt-range_bar.pkl')
df1

In [ ]:
df2 = pd.read_pickle(f'{base_path}btcusdt-range_bars_with_indicators.pkl')
df2.tail(4)

In [ ]:
# target = pd.to_datetime('2023-04-29 13:17:00.999') 
# # - pd.Timedelta(minutes=10)
# window_df = df1.loc[df1.index >= target]
# print(len(window_df))
# window_df

In [ ]:
#double the length
window_df = df1[-104:]
print(len(window_df))
window_df

In [ ]:
# Calculate the difference between the current close and the last close
close_diff = window_df['close'] - window_df['close'].shift(1)
range_size = window_df['average_adr'] * 0.1
window_df['range_size'] = range_size
# Add the difference as a new column 'close_diff'
window_df['close_diff'] = close_diff
window_df

### Rang Bar Creation Is Flawed
The range_size should be equal to the close_diff, here we can see it's not. If the range_size is not achieved the klines should be bufferred and aggr until it is. Thta's not happening in the code

In [ ]:
df_indicators = window_df.copy()

In [ ]:
# calculate MACD with custom parameters
macd = ta.trend.MACD(df_indicators['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
# add MACD values to the dataframe as new columns
df_indicators['macd'] = macd.macd()
df_indicators['macd_signal'] = macd.macd_signal()
df_indicators['macd_histogram'] = macd.macd_diff()
df_indicators

## 26 Bars

2023-04-29 13:34:59.999	29401.0	29485.1	29499.0	29350.0	373.498	1351.0	94.123643 | -2.926256	NaN	NaN (with fillna=False)

2023-04-29 13:34:59.999	29401.0	29485.1	29499.0	29350.0	373.498	1351.0	94.123643 | -2.926256 0.673993 -3.600249 (with fillna=True)

## 52 Bars

2023-04-29 13:34:59.999	29401.0	29485.1	29499.0	29350.0	373.498	1351.0	94.123643 |	-8.325256	-6.782518	-1.542737 (fillna=False)

2023-04-29 13:34:59.999	29401.0	29485.1	29499.0	29350.0	373.498	1351.0	94.123643 |	-8.325256	-6.781790	-1.543466 (with fillna=True)

## 104 Bars (fillna=True)

2023-04-29 13:34:59.999	29401.0	29485.1	29499.0	29350.0	373.498	1351.0	94.123643 |	-4.907634	-1.822050	-3.085583

# MACD Issue Theory

We can see in the case of fillna=False that this indicator uses the values from before, in it's windows, how much data is need for the values to stabilize surely there must be some info on this